In [680]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd

In [681]:
#parameters
scan_id=3
sample_rate=10
no_of_lines =50
line_tolarance=1.5
model_name="svm"

In [682]:
def process_data(file, sample_rate = sample_rate):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    point_array_sampled = []
    for i in range(0,len(coord),sample_rate):
        point_array_sampled.append(coord[i])
        
    point_array_sampled = np.array(point_array_sampled)
        
    return np.array( point_array_sampled)

In [683]:
def plot_graph(point_data,outline_points=None, lines=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create cubic bounding box to simulate equal aspect ratio
    X = point_data[:,0]
    Y = point_data[:,1]
    Z = point_data[:,2]

    ax.scatter(point_data[:,0], point_data[:,1], point_data[:,2], s=1)

    #bounding box to show foot in scale
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())

    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')
        
    if (outline_points is not None):
        flatten = outline_points.reshape(no_of_lines*2,3)
        ax.scatter(flatten[:,0], flatten[:,1], flatten[:,2], s=500/no_of_lines, color='red')

    plt.grid()
    plt.show()

In [684]:
def project_points(point_data, axis=1, height_limit=100000): #axis values x=0/y=1/z=2 implement limit
    if(axis==0):
        point_array[:,0]=0
    if(axis==1):
        point_array[:,1]=0
    if(axis==2):
        point_array[:,2]=0
        
    return point_array

In [685]:
def get_outline(point_data, no_of_lines = no_of_lines, line_tolarance = line_tolarance):
    
    z_min = min(point_data[:,2])
    z_max =max(point_data[:,2])
    
    step = (z_max-z_min)/(no_of_lines+1)
    
    lines = np.arange(z_min, z_max, step)[1:]
    
    outline_points=[]
    
    for line in lines:
        line_points=[]
        candidates = point_data[np.where(np.logical_and(point_data[:,2]>line, point_data[:,2]<line+line_tolarance))]
        
        if(len(candidates)==0 or len(outline_points)==no_of_lines):
            break
            
        line_points.append(candidates[np.where(candidates[:,0]==min(candidates[:,0]))][0])
        line_points.append(candidates[np.where(candidates[:,0]==max(candidates[:,0]))][0])
        
        
        outline_points.append(line_points)
        
    return np.array(outline_points).reshape((no_of_lines,2,3)),lines

In [774]:
def read_dataset():
    all_set = pd.read_csv("Datasets/all_data.csv", sep=',',header=0)
    train_set=pd.read_csv("Datasets/train_set.csv", sep=',',header=0)
    test_set=pd.read_csv("Datasets/test_set.csv", sep=',',header=0)
    
    return all_set.drop(all_set.index[[0,1]]),train_set.drop(train_set.index[[0,1]]), test_set.drop(test_set.index[[0,1]])

In [775]:
def load_model(model=model_name):
    if(model=="svm"):
        return pickle.load(open("Models/SVM_Classifier.pkcls", 'rb'))
    else:
        return

In [776]:
points = open("Datasets/Foot Models/points/left/3/sub473ll.pts")
point_array = process_data(points)

In [777]:
outline =get_outline(project_points(point_array))
outline_points= outline[0]
lines=outline[1]

In [778]:
dataset= read_dataset()
all_data= dataset[0]
train_set = dataset[1]
test_set = dataset[2]

In [759]:
model=load_model() #implement classification model
test_features=test_set[["No.","Cluster","Foot Length","Foot Breadth 1","Short Heel Girth"]]

In [779]:
all_data

,Foot Length,Arch Length,Heel to 5th Toe,Foot Breadth 1,Foot Breadth 2,Heel Breadth,Medial Malleolus Height,Medial Malleolus Distance,Lateral Malleolus Height,Lateral Malleolus Distance,...,Mid Foot Breadth,Height at 50% FL,Ball Girth,Instep Girth,Long Heel Girth,Short Heel Girth,Ankle Girth,Waist Girth,Cluster,No.
2,279.0,180.39,279.0,105.98,110.09,68.98,25.94,179.95,43.7,141.17,...,102.41,68.27,272.47,279.34,372.88,344.86,273.16,272.31,C1,sub1
3,251.63,180.13,208.59,99.03,99.03,64.01,61.54,128.15,47.77,126.32,...,90.57,71.82,242.24,259.05,341.82,322.52,246.12,253.33,C5,sub2
4,269.92,199.26,242.3,92.28,92.08,64.42,65.55,137.97,48.79,101.88,...,83.84,69.58,228.0,243.14,358.1,331.04,252.62,231.24,C5,sub3
5,268.08,189.99,220.53,105.14,104.91,66.06,55.3,135.6,42.06,133.98,...,98.72,64.95,253.99,270.17,340.13,329.55,261.72,254.82,C5,sub4
6,260.58,183.7,220.27,107.83,107.83,65.37,57.65,132.63,46.93,129.22,...,98.61,70.09,259.31,274.35,354.03,336.74,262.88,262.6,C5,sub5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,226.74,114.24,194.68,92.39,82.13,58.69,35.93,167.46,0.42,198.33,...,79.25,62.79,420.76,230.18,306.57,300.05,234.43,252.76,C1,sub200
197,255.76,188.22,236.15,96.27,96.27,70.7,55.06,132.07,45.59,129.81,...,90.29,69.17,235.46,245.62,364.27,329.57,244.96,238.65,C5,sub201
198,244.65,178.87,197.86,91.93,91.79,61.32,54.82,122.08,45.01,124.24,...,87.92,62.17,224.18,243.21,319.88,307.65,243.38,232.53,C4,sub202
199,253.09,181.8,202.36,94.12,93.33,63.29,53.08,123.46,48.75,126.25,...,88.98,61.58,234.76,240.23,345.05,314.57,251.22,231.19,C4,sub203


In [767]:
model.predict([[1,1,0]])

(array([4.]),
 array([[0.06428333, 0.00531836, 0.07592739, 0.01657941, 0.83789151]]))

In [762]:
model.predict([[1,1,1]])

(array([4.]),
 array([[0.06265993, 0.00578155, 0.09134616, 0.01420377, 0.82600859]]))